<img src="../images/vegan-logo-resized.png" style="float: right; margin: 10px;">

# Data Collection

Author: Gifford Tompkins

---

Project 03 | Notebook 0 of 6

# Objective
The objective of this notebook is to scrape a substantial body of posts from the two subreddits [r/Vegan](https://www.reddit.com/r/vegan/) and [r/Vegetarian](https://www.reddit.com/r/vegetarian/) and to save them into a CSV file for preprocessing and data cleaning, to be completed in the next notebook.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Collection" data-toc-modified-id="Data-Collection-1">Data Collection</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-2">Objective</a></span></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-3">Import Libraries</a></span></li><li><span><a href="#Collecting-Data-Using-pushshift.io" data-toc-modified-id="Collecting-Data-Using-pushshift.io-4">Collecting Data Using <em>pushshift.io</em></a></span><ul class="toc-item"><li><span><a href="#Creating-the-corpus" data-toc-modified-id="Creating-the-corpus-4.1">Creating the corpus</a></span></li><li><span><a href="#Confirm-the-corpus" data-toc-modified-id="Confirm-the-corpus-4.2">Confirm the corpus</a></span></li></ul></li><li><span><a href="#Summary-and-Conclusion" data-toc-modified-id="Summary-and-Conclusion-5">Summary and Conclusion</a></span></li></ul></div>

# Import Libraries 

In [2]:
import pandas as pd
import requests
import numpy as np
import time
from datetime import datetime, timedelta

In [5]:
# Custom functions saved in repository
from project_functions.data_collection import get_data_json, create_doc, append_to_corpus    

# Collecting Data Using _pushshift.io_

We will be collecting posts (or documents, as they will be referred to in this notebook) through the [pushshift.io API](https://pushshift.io/). For that we will submit a base `url` with a dictionary of parameters. One of those parameters being the name of the subbreddit, either `r/Vegan` or `r/Vegetarian`.

In [3]:
url = 'https://api.pushshift.io/reddit/submission/search'
subreddits = ['Vegetarian','Vegan']  
corpus_csv = '../data/api_data.csv'

## Creating the corpus
We will create an empty dataframe to be saved as our `api_data.csv` that we can append each new batch of documents onto. Since we will be doing multiple pulls and hopefully getting up to 20,000 rows of data, we would like to save our data as we go in the event that our connection breaks or our function or program times out.

For custom functions used in this section, see the [data collection code](./project_functions/data_collection.py).

In [4]:
# Create empty dataframe to store our corpus as a csv file
df = pd.DataFrame(columns=['title','selftext','vegan'])
df.to_csv(corpus_csv,index=False)

In [63]:
# Loop through the two subreddits and pull the document batches with the pushift.io api.
for i, sub in enumerate(subreddits):
    
    # Instantiate and define the url parameters dictionary.
    url_params = {
        'subreddit': sub,
        'size': 500
    }
    
    # Pull 20 batches of posts for each subreddit.
    for count in range(20):
        
        # Pull data from json
        posts = get_data_json(count, url, url_params)

        # Instantiate the bath of documents as a list item
        documents = []
        
        for post in posts:
            
            # Create a document from the post using the custom function create_doc
            # Classify the post as Vegan post: 1 or a Vegetarian post: 0
            document = create_doc(i, post)
            
            # Append the document onto the documents batch
            documents.append(document)
        
        # Append the batch of documents to the corpus using the custom function append_to_corpus
        append_to_corpus(documents, corpus_csv)
        
        # Update 'before' paramter in url_params before running next batch.
        url_params['before'] = data[-1]['created_utc']

        # Put a delay on our loop to slow down api usage
        time.sleep(10)
        
    # Remove the before parameter so that it can be redefined in the next subreddit
    url_params.pop('before')

r/Vegetarian: Pull number 1
https://api.pushshift.io/reddit/submission/search?subreddit=Vegetarian&size=500
500 documents appended to corpus.
r/Vegetarian: Pull number 2
https://api.pushshift.io/reddit/submission/search?subreddit=Vegetarian&size=500&before=1570031340
500 documents appended to corpus.
r/Vegetarian: Pull number 3
https://api.pushshift.io/reddit/submission/search?subreddit=Vegetarian&size=500&before=1568814615
500 documents appended to corpus.
r/Vegetarian: Pull number 4
https://api.pushshift.io/reddit/submission/search?subreddit=Vegetarian&size=500&before=1567437169
Post: Going vegetarian is missing selftext. Appending title only.
{'title': 'Going vegetarian', 'vegan': 0, 'selftext': None}
500 documents appended to corpus.
r/Vegetarian: Pull number 5
https://api.pushshift.io/reddit/submission/search?subreddit=Vegetarian&size=500&before=1566162390
500 documents appended to corpus.
r/Vegetarian: Pull number 6
https://api.pushshift.io/reddit/submission/search?subreddit=Vege

## Confirm the corpus
Confirm that the posts have been pulled and that the data looks appropriate. We will do more extensive data-cleaning and EDA in the next notebook.

In [6]:
data = pd.read_csv('../data/api_data.csv')

In [7]:
data.head()

,title,selftext,vegan
0,Lentil soup with sliced Beyond Meat sausage ta...,NaN,0
1,Lentil soup with sliced Beyond Meat sausage ta...,NaN,0
2,Recipes for thanksgiving?,Hi! My bf’s sister is hosting thanksgiving thi...,0
3,What Happens IMMEDIATELY After You Die?? The T...,[https://www.youtube.com/watch?v=Dyvl\_8SoAjc...,0
4,Vegetarian Chili for a cold October evening,NaN,0


In [8]:
data.tail()

,title,selftext,vegan
20995,Curious if you guys think this is a step in th...,NaN,1
20996,The truth,NaN,1
20997,I am sorry guys.,NaN,1
20998,"I call this a “Sloppy Mel”- beyond meat, Jones...",NaN,1
20999,Can I suggest any Facebook users do one small ...,Okay this isn't real activism and it's okay to...,1


In [9]:
data['vegan'].value_counts()

1    10500
0    10500
Name: vegan, dtype: int64

In [67]:
train.shape

(21000, 3)

In [11]:
data.isnull().mean()

title       0.000
selftext    0.571
vegan       0.000
dtype: float64

**Observations:**
> Appears to be some duplicates and the selftext column has selveral null values, but all items are stored properly in the [`corpus.csv`](./data/api_data.csv). The first posts are vegetarian while last posts are vegan and we have pulled equal numbers of posts from each subreddit.  
>
> The corpus looks ready to go.

# Summary and Conclusion
We have gathered a solid body of data for our NLP analysis. We did this through submitting batch request through the pushift.io API. We then stored the data of interest into a corpus that we saved as a CSV.

In the next notebook we will clean our data and do some initial exploratory analysis. After that we will have a better sense of direction for building our Vegan classification model.